In [1]:
import os
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_model_analysis as tfma

from tfx import components
from tfx.proto import example_gen_pb2, trainer_pb2,pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

In [2]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "movies-rec"
SERVING_MODEL_DIR = "serving_model_dir"

In [3]:
DATA_ROOT = "data/movie"

In [4]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [5]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
    ]),
    
)

example_gen = components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output
)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
statistics_gen = components.StatisticsGen(
    examples=example_gen.outputs["examples"]
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [7]:
context.show(statistics_gen.outputs["statistics"])

In [8]:
schema_gen = components.SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [9]:
example_validator = components.ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [10]:
context.show(example_validator.outputs["anomalies"])

In [11]:
TRANSFORM_MODULE_FILE = "movie_transform.py"

In [12]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

FEATURE_KEYS = [
    "title"
    "genres"
]

def transformed_name(key):
    return f"{key}_xf"

def preprocessing_fn(inputs):
    outputs = {}
    
    for key in FEATURE_KEYS:
        outputs[transformed_name(key)] = tf.strings.lower(
            inputs[key]
        )
    
    return outputs

Overwriting movie_transform.py


In [13]:
transform = components.Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
context.run(transform)

KeyError: 'titlegenres'